In [1]:
import numpy as np
import torch
import scipy.stats as scs
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn
import torch.optim as optim
import torch.distributions as dist
from pyhmc import hmc
from tqdm import tqdm

In [2]:
import ot
def W2(x,y):
    return torch.sqrt(ot.emd2(torch.ones(x.shape[0])/x.shape[0], torch.ones(y.shape[0])/y.shape[0], ot.dist(x, y)))


c:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\ot\backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


In [3]:
# load all mnist data
import torchvision.datasets as datasets
import torchvision.transforms as transforms

np.random.seed(990109)
torch.manual_seed(990109)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([transforms.ToTensor(),transforms.Lambda(lambda x: x.view(784))])
trainset = datasets.MNIST(root='../MNIST_exp', train=True, download=False, transform=transform)
data_true = trainset.data.numpy()
data_true = data_true.reshape(data_true.shape[0], -1)

# Add Gaussian noise to dequentize
noise = scs.norm.rvs(0, 0.05, data_true.shape)
X_noisy_flat = (data_true + noise)

# Apply ECDF transformation
X_ecdf = np.zeros_like(X_noisy_flat)
ecdf_list = []
for dim in (range(X_noisy_flat.shape[1])):
    ecdf = ECDF(X_noisy_flat[:, dim])
    ecdf_list.append(ecdf)
    X_ecdf[:, dim] = np.clip(ecdf(X_noisy_flat[:, dim]), 1e-6, 1 - 1e-6)



# Apply inverse of standard normal CDF (ppf)
X_gaussian = scs.norm.ppf(X_ecdf).reshape(-1, 28,28)
y_gaussian = torch.ones(X_gaussian.shape[0], dtype=torch.long)
# make it a tensor with shape (n_samples, n_channels, height, width)
X_gaussian = torch.tensor(X_gaussian, dtype=torch.float32).unsqueeze(1)  # Add channel dimension

# Split the data into training and testing sets (50/50 split)
X_train, X_test, y_train, y_test = train_test_split(X_gaussian, y_gaussian, test_size=0.5, random_state=42)
# Create TensorDataset objects
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


c:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


MemoryError: Unable to allocate 359. MiB for an array with shape (47040000,) and data type float64

In [ ]:

Gauss_LL_mnist = []
Gauss_W2_mnist = []
for i in tqdm(range(10)):

    X_train, X_test, y_train, y_test = train_test_split(X_gaussian, y_gaussian, test_size=0.5, random_state=i)


    gauss_cov = np.cov(X_train.reshape(-1, 784).T)
    top = (scs.multivariate_normal.logpdf(X_test.reshape(-1, 784), mean=np.zeros(784), cov=gauss_cov+1e-10*np.eye(784)).mean())
    bot = (scs.multivariate_normal.logpdf(X_test.reshape(-1, 784), mean=np.zeros(784), cov=np.eye(784)).mean())
    Gauss_LL_mnist.append(top-bot)
    sims_gauss = scs.multivariate_normal.rvs(mean=np.zeros(784), cov=gauss_cov+1e-10*np.eye(784), size=25)
    Gauss_W2_mnist.append(W2(X_test.reshape(-1,28*28).float(),torch.tensor(sims_gauss).reshape(-1,28*28).float()))

100%|██████████| 1/1 [00:06<00:00,  6.80s/it]


In [ ]:
np.mean(Gauss_LL_mnist), np.std(Gauss_LL_mnist),np.mean(Gauss_W2_mnist), np.std(Gauss_W2_mnist)

(126.42638961130842, 0.0, 37.04207, 0.0)

In [26]:
GGCNN_LL_mnist = [
197.11,
203.69,
220.88,
217.37,
164.36,
196.70,
168.83,
194.59,
221.96]






np.mean(GGCNN_LL_mnist), np.std(GGCNN_LL_mnist)


c:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(198.38777777777779, 19.72151773389652, nan, nan)

# IGC

In [6]:

    #### IGC model



    class SoftRank(nn.Module):
        """Differentiable ranking layer"""
        def __init__(self, alpha=1000.0):
            super(SoftRank, self).__init__()
            self.alpha = alpha # constant for scaling the sigmoid to approximate sign function, larger values ensure better ranking, overflow is handled properly by PyTorch

        def forward(self, inputs):
            # input is a ?xSxD tensor, we wish to rank the S samples in each dimension per each batch
            # output is  ?xSxD tensor where for each dimension the entries are (rank-0.5)/N_rank
            x = inputs.unsqueeze(-1) #(?,S,D) -> (?,S,D,1)
            x_2 = x.repeat(1, 1, 1, x.shape[1]) # (?,S,D,1) -> (?,S,D,S) (samples are repeated along axis 3, i.e. the last axis)
            x_1 = x_2.transpose(1, 3) #  (?,S,D,S) -> (?,S,D,S) (samples are repeated along axis 1)
            return torch.transpose(torch.sum(torch.sigmoid(self.alpha*(x_1-x_2)), dim=1), 1, 2)/(torch.tensor(x.shape[1], dtype=torch.float32))


    class IGC(nn.Module):
        
        def __init__(self, hidden_size=100, layers_number=2, output_size=2):
            super(IGC, self).__init__()
            self.dim_latent = 3 * output_size
            self.hidden_size = hidden_size
            self.layers_nuber = layers_number
            self.output_size = output_size
            self.linear_in = nn.Linear(in_features=self.dim_latent, out_features=self.hidden_size) 
            self.linear = nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
            self.linear_out = nn.Linear(in_features=self.hidden_size, out_features=self.output_size)
            self.marginal_cdfs = None
            self.ecdf_10e6_samples = None

        def forward_train(self, z):
            '''
            Input noise z with shape (M,dim_latent)\\
            Outputs (u,v) pairs with shape (M,output_size=2), while ensuring u and v each have uniform marginals.
            '''
            y = torch.relu(self.linear_in(z))
            for layer in range(self.layers_nuber):
                y = torch.relu(self.linear(y))
            y = self.linear_out(y).unsqueeze(0)
            u = SoftRank()(y).squeeze(0)
            return u
            
        def Energy_Score_pytorch(self,beta, observations_y, simulations_Y):
            n = len(observations_y)
            m = len(simulations_Y)

            # First part |Y-y|. Gives the L2 dist scaled by power beta. Is a vector of length n/one value per location.
            diff_Y_y = torch.pow(
                torch.norm(
                    (observations_y.unsqueeze(1) -
                    simulations_Y.unsqueeze(0)).float(),
                    dim=2,keepdim=True).reshape(-1,1),
                beta)

            # Second part |Y-Y'|. 2* because pdist counts only once.
            diff_Y_Y = 2 * torch.pow(
                nn.functional.pdist(simulations_Y),
                beta)
            Energy = 2 * torch.mean(diff_Y_y) - torch.sum(diff_Y_Y) / (m * (m - 1))
            return Energy


        def forward(self, n_samples):
            ''' 
            Function to sample from the copula, once training is done.

            Input: n_samples - number of samples to generate
            Output: torch.tensor of shape (n_samples, output_size) on copula space.
            '''
            with torch.no_grad():
                if self.marginal_cdfs is None:
                    self.marginal_cdfs = []
                    # sample 10^6 points from the latent space and compute empirical marginal cdfs
                    z = torch.randn(10**6, self.dim_latent)
                    y = torch.relu(self.linear_in(z))
                    for layer in range(self.layers_nuber):
                        y = torch.relu(self.linear(y))
                    y = self.linear_out(y) # samples used to approximate cdfs
                    for dim in range(y.shape[1]):
                        ecdf = ECDF(y[:, dim].numpy())
                        self.marginal_cdfs.append(ecdf)
                    self.ecdf_10e6_samples = y
                # sample the latent space and apply ecdfs
                z = torch.randn(n_samples, self.dim_latent)
                y = torch.relu(self.linear_in(z))
                for layer in range(self.layers_nuber):
                    y = torch.relu(self.linear(y))
                y = self.linear_out(y)
                for dim in range(y.shape[1]):
                    y[:, dim] = torch.tensor(self.marginal_cdfs[dim](y[:, dim].numpy()), dtype=torch.float32)
                return y



In [7]:
# Calculate the number of parameters
igc_cop = IGC(hidden_size=100, layers_number=2, output_size=28*28)
total_params = sum(p.numel() for p in igc_cop.parameters())
print(f'Total parameters: {total_params}')

Total parameters: 324584


In [50]:
for seed in range(10):
    # Split the data into training and testing sets (50/50 split)
    X_train, X_test, y_train, y_test = train_test_split(X_gaussian, y_gaussian, test_size=0.5, random_state=seed)
    # make training data on 0-1 scale
    X_train_cop = torch.tensor(scs.norm.cdf(X_train.reshape(-1,28*28)),dtype=torch.float32).clip(1e-5,1-1e-5)
    
    # training loop
    igc_cop = IGC(hidden_size=100, layers_number=3, output_size=28*28)

    u_obs = X_train_cop

    optimizer = torch.optim.Adam(igc_cop.parameters())
    loss_hist = []

    for i in tqdm(range(1000)):
        optimizer.zero_grad()
        u = igc_cop.forward_train(torch.randn((200, igc_cop.dim_latent)))
        loss = igc_cop.Energy_Score_pytorch(1, u_obs[np.random.choice(range(u_obs.shape[0]),100,replace=True)], u)
        loss.backward()
        optimizer.step()
        loss_hist.append(loss.item())
        if i % 100 == 0:
            print(loss.item())

    samples_cdf = igc_cop.forward_train(torch.randn((25, igc_cop.dim_latent)))
    print(seed)
    print((W2(X_test.reshape(-1,28*28).float(),torch.tensor(scs.norm.ppf(samples_cdf.detach().numpy())).float(),),'IGC'))

    torch.save(igc_cop.state_dict(), f'igc_cop_mnist_10runs_seed{seed}.pth')



  0%|          | 1/501 [00:01<09:06,  1.09s/it]

11.488859176635742


 20%|██        | 101/501 [01:42<07:26,  1.12s/it]

11.326433181762695


 40%|████      | 201/501 [03:24<05:14,  1.05s/it]

...

 80%|████████  | 401/501 [06:43<01:37,  1.03it/s]

11.436563491821289


100%|██████████| 501/501 [08:22<00:00,  1.00s/it]

11.369071960449219
9


(tensor(35.8969), 'IGC')


In [10]:
samples_cdf = igc_cop.forward_train(torch.randn((25, igc_cop.dim_latent)))
print((W2(X_test.reshape(-1,28*28).float(),torch.tensor(scs.norm.ppf(samples_cdf.detach().numpy())).float(),),'IGC'))


NameError: name 'X_test' is not defined

In [11]:

IGC_MNIST_W2 = [
    36.0713,
    36.0503,
    36.1236,
    35.9470,
    36.0018,
    36.0666,
    35.9763,
    35.9971,
    36.0689,
    35.8969
]
np.mean(IGC_MNIST_W2), np.std(IGC_MNIST_W2)

(36.019980000000004, 0.0648794697882153)

In [6]:

vine_MNIST_LL=[
182.8999548028951,
183.07594252548503,
183.4142022677527,
182.9113776227081,
182.32917378065636,
183.2038983816669,
182.65361634253821,
183.54003720756043,
183.40584870213985,
182.52377025599756]
np.mean(vine_MNIST_LL), np.std(vine_MNIST_LL)

(182.99578218894004, 0.3859699447004303)